1. Read *Wikipedia.csv* as written via Wikipedia class.
2. Tokenize the text, and save tokens into *df_token*.
3. Build similarity matrix and write this to file *Wikipedia-sims.csv* with columns:  
    id : *article id*  
    similar_articles : *list of article ids*


In [1]:
import sys, os
import pandas as pd

# Read data

Read .csv files and make a pandas dataframe.

In [2]:
actual_dir=os.getcwd()
os.chdir('../data')
csvfile1 = 'Wikipedia-dog.csv'
csvfile2 = 'Wikipedia-fish.csv'

df=pd.read_csv(csvfile1,index_col=0)
df2=pd.read_csv(csvfile2,index_col=0)
df=df.append(df2)
del df2

df=df[df['page_type']=='article']


del df['page_type']
del df['text_raw']

df.columns=['url','title']

df.tail(5)




,url,title
43590845,https://en.wikipedia.org/wiki/Whale_feces,Whale feces
42614454,https://en.wikipedia.org/wiki/Whale_watching_i...,Whale watching in New Zealand
49735416,https://en.wikipedia.org/wiki/Tail_sailing,Tail sailing
52243894,https://en.wikipedia.org/wiki/Bubble_net_feeding,Bubble net feeding
53720250,https://en.wikipedia.org/wiki/Ethelbert_(whale),Ethelbert (whale)


In [3]:

model='LSI'


if ( model == 'LSI' ):
    df_sims=pd.read_csv('Wikipedia-sims-lsi.csv',index_col=0)

elif (model == 'LDA' ):
    df_sims=pd.read_csv('Wikipedia-sims-lda.csv',index_col=0)

df_sims.head()

,similar_articles,scores
1467938,"[8886648, 51676708, 17430047, 22345050, 373313...","[0.34231352806091309, 0.30013024806976318, 0.1..."
275388,"[4269567, 17430047, 22345050, 22078621, 330409...","[0.16771674156188965, 0.14842286705970764, 0.1..."
2352562,"[2676271, 4269567, 5141410, 5740890, 3304092, ...","[0.28183633089065552, 0.23679673671722412, 0.2..."
17021807,"[17430047, 37331361, 4269567, 3304092, 5141410...","[0.36869356036186218, 0.2088133692741394, 0.19..."
20777185,"[4269567, 19282291, 5141410, 3279728, 17862013...","[0.2432430237531662, 0.23855295777320862, 0.22..."


In [4]:
from ast import literal_eval
import numpy as np

# Put titles into a dataframe
df_titles=df['title']
df_urls=df['url']

# Add empty columns to Pandas dataframe:
df['other titles']=""
df['other urls']=""
df['scores']=""

# Change similar_articles from ids, to titles

for article_id in df.index:
    other_articles_ids=df_sims.loc[article_id]['similar_articles']
    other_articles_scores=df_sims.loc[article_id]['scores']
    # Take only top 5 article ids and scores
    other_articles_ids=literal_eval(other_articles_ids)[:5]
    other_articles_scores=literal_eval(other_articles_scores)[:5]
    # Round scores, and convert numpy array to list
    other_articles_scores=np.round(other_articles_scores,2).tolist()
    # Select titles in other_articles_ids, and convert to list
    other_titles=df_titles.loc[other_articles_ids].values.tolist()
    # Select urls in other_articles_ids, and convert to list
    other_urls=df_urls.loc[other_articles_ids].values.tolist()
    
    # Fill DF entries:
    df['other titles'][article_id]=other_titles
    df['scores'][article_id]=other_articles_scores
    df['other urls'][article_id]=other_urls
df.head()

,url,title,other titles,other urls,scores
1467938,https://en.wikipedia.org/wiki/Kennel,Kennel,"[Cattery, Indian National Kennel Club, Breed t...","[https://en.wikipedia.org/wiki/Cattery, https:...","[0.34, 0.3, 0.19, 0.15, 0.14]"
275388,https://en.wikipedia.org/wiki/Cynology,Cynology,"[Dog, Breed type (dog), Dog World (newspaper),...","[https://en.wikipedia.org/wiki/Dog, https://en...","[0.17, 0.15, 0.12, 0.12, 0.12]"
2352562,https://en.wikipedia.org/wiki/Pack_(canine),Pack (canine),"[Canid hybrid, Dog, Origin of the domestic dog...","[https://en.wikipedia.org/wiki/Canid_hybrid, h...","[0.28, 0.24, 0.23, 0.16, 0.1]"
17021807,https://en.wikipedia.org/wiki/Rare_breed_(dog),Rare breed (dog),"[Breed type (dog), Lists of dogs, Dog, Dog bit...",[https://en.wikipedia.org/wiki/Breed_type_(dog...,"[0.37, 0.21, 0.19, 0.17, 0.13]"
20777185,https://en.wikipedia.org/wiki/Dogs_in_ancient_...,Dogs in ancient China,"[Dog, Dogs in Mesoamerica, Origin of the domes...","[https://en.wikipedia.org/wiki/Dog, https://en...","[0.24, 0.24, 0.22, 0.21, 0.2]"


In [5]:
if ( model == 'LSI' ):
    df.to_csv('Wikipedia-lsi.csv')
elif ( model == 'LDA' ):
    df.to_csv('Wikipedia-lda.csv')
